# Testing Deferred Revenue in Python
Will this be easier for everyone to use than Matlab?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Processing Base Billings Data

In [2]:
df = pd.read_excel('../data/all_billings_inputs.xlsx', sheet_name='base_billings')

In [3]:
df.sample(10)

,Document Currency,Enterprise BU Desc,Frequency,Invoice Fiscal Year Period Desc,Product Config Type,Rev Rec Category,Rule For Bill Date,Sales Type,Subscription Term,Completed Sales ( DC ),Completed Sales
6870,CAD,Experience Cloud,ONGO,2018-10,ONORE,D,Y1,DEFERRED,0,32087.50,24894.67
3170,AUD,Experience Cloud,1TME,2018-11,NaN,NaN,NaN,RECOGNIZED,0,6781.93,4893.61
18070,EUR,Experience Cloud,ONGO,2018-10,1Y,D,Y1,DEFERRED,12,23124.98,26972.14
2813,AUD,Experience Cloud,1TME,2015-11,NaN,NaN,NaN,RECOGNIZED,0,-28003.37,-20572.74
13808,EUR,Creative,ONGO,2015-10,1Y,D,YA,DEFERRED,0,109953.60,124950.52
27165,JPY,Creative,NaN,2017-05,1Y,NaN,NaN,DEFERRED,0,0.00,0.00
3702,AUD,Experience Cloud,ONGO,2015-12,OENSV,D,YQ,DEFERRED,0,2499.99,1769.44
47531,USD,Experience Cloud,ONGO,2019-09,ONORE,D,YC,DEFERRED,0,850002.02,850002.02
45747,USD,Experience Cloud,ONGO,2016-01,ONORE,NaN,NaN,RECOGNIZED,0,0.00,0.00
7694,CHF,Creative,NaN,2018-06,1Y,D,YA,DEFERRED,12,1016009.40,1018853.15


## Changing the column names early since they are inconsistent across other reports

In [4]:
df.columns

Index(['Document Currency', 'Enterprise BU Desc', 'Frequency',
       'Invoice Fiscal Year Period Desc', 'Product Config Type',
       'Rev Rec Category', 'Rule For Bill Date', 'Sales Type',
       'Subscription Term', 'Completed Sales ( DC )', 'Completed Sales'],
      dtype='object')

In [20]:
df.rename(index = str, columns = {'Document Currency': 'curr',
                                 'Enterprise BU Desc': 'BU',
                                 'Invoice Fiscal Year Period Desc': 'period',
                                 'Product Config Type': 'config',
                                 'Rev Rec Category': 'rev_req_type',
                                 'Rule For Bill Date': 'rebill_rule',
                                 'Completed Sales ( DC )': 'DC_amount',
                                 'Completed Sales': 'US_amount'}, inplace=True)

In [21]:
df.columns

Index(['curr', 'BU', 'Frequency', 'period', 'config', 'rev_req_type',
       'rebill_rule', 'Sales Type', 'Subscription Term', 'DC_amount',
       'US_amount'],
      dtype='object')

## Filter that removes any currency that has  < 10 transactions. 


In [7]:
# creates a list of the currencies and the number of transactions for each currency
vc = df['curr'].value_counts()
print(vc)

USD    12427
EUR     7725
GBP     5714
AUD     5068
JPY     4919
CHF     2389
SEK     2260
DKK     2172
NOK     1854
CAD     1520
HKD      486
BRL      466
RUB      461
KRW      247
CLP      211
ARS      210
COP      210
SGD      209
PEN      190
INR      181
PHP      151
TWD      148
THB      144
MYR      138
IDR      132
NZD       67
ILS       38
TRY       27
SAR        4
BMD        2
AED        1
MXP        1
Name: curr, dtype: int64


In [8]:
# Create variable that is true if the number of transaction is greater than 10, false otherwise
keep_these = vc.values > 10
# filtering only currencies that were greater than 10
keep_curr = vc[keep_these]
a = keep_curr.index
# filtering the dataframe to remove any of teh currencies not in our list
df = df[df['curr'].isin(a)]

## Just keeping track of the currencies we removed in our model_dict data structure

In [9]:
remove_these = vc[vc.values <= 10].index
model_dict = {'curr_removed': list(vc[remove_these].index)}
delete_curr = list(remove_these)

## The FX database does not have information on the following currencies
 - AED (United Arab Emirates Dirham)
 - BMD (Bermudan Dollar)
 - MXP (Mexican Peso)
 - TRY (Turkish Lira)

In [10]:
if 'TRY' not in model_dict:
    model_dict['curr_removed'].append('TRY')
    delete_curr.append('TRY')
    a = a.drop('TRY')
    
print('Model dictionary', model_dict)
print('Deleted Currencies', delete_curr)

Model dictionary {'curr_removed': ['SAR', 'BMD', 'AED', 'MXP', 'TRY']}
Deleted Currencies ['SAR', 'BMD', 'AED', 'MXP', 'TRY']


In [11]:
print("---Removing infrequent currencies from billings history---")
print('Total number of currencies in the base billings file: ', len(vc))
if len(model_dict['curr_removed'])==0:
    print('No currencies were removed, all contained 10 or more billings')
    print('Currencies in the base billings file')
    for item in a:
        print(a[item], end = " ")
else:
    print('\n Currencies were removed: ', len(model_dict['curr_removed']))

    for item in remove_these:
        print(item, ', ', end="")
        
    print("\n\n{} Remaining currencies: ".format(len(a)))
    for item in a:
        print(item, ', ', end="")

---Removing infrequent currencies from billings history---
Total number of currencies in the base billings file:  32

 Currencies were removed:  5
SAR , BMD , AED , MXP , 

27 Remaining currencies: 
USD , EUR , GBP , AUD , JPY , CHF , SEK , DKK , NOK , CAD , HKD , BRL , RUB , KRW , CLP , ARS , COP , SGD , PEN , INR , PHP , TWD , THB , MYR , IDR , NZD , ILS , 

# Removing any of the values that are zero

In [13]:
print('This is the length of the dataframe before removing zeros: ', len(df))
df = df[df['DC_amount']!=0]
print('This is the length of the dataframe after removing zeros: ', len(df))

This is the length of the dataframe before removing zeros:  49764
This is the length of the dataframe after removing zeros:  46285


In [14]:
df.sample(10)

,curr,Enterprise BU Desc,Frequency,period,config,rev_req_type,rebill_rule,Sales Type,Subscription Term,DC_amount,US_amount
44536,USD,Experience Cloud,1TME,2019-06,ONORE,D,YH,DEFERRED,0,-5249.62,-5249.62
23974,GBP,Experience Cloud,ONGO,2017-01,1Y,D,YQ,DEFERRED,12,8988.19,9906.01
25992,HKD,Document Cloud,NaN,2018-10,NaN,A,NaN,DEFERRED,12,-1222.34,-155.77
9914,COP,Creative,NaN,2019-10,NaN,NaN,NaN,RECOGNIZED,0,4050103.00,1192.42
40870,USD,Document Cloud,NaN,2017-08,1Y,D,YQ,DEFERRED,0,94680.00,94680.00
38822,USD,Creative,NaN,2019-02,1Y,D,NaN,DEFERRED,1,-514380.47,-514380.47
46495,USD,Experience Cloud,ONGO,2017-09,OCONS,D,YQ,DEFERRED,0,98250.00,98250.00
44636,USD,Experience Cloud,1TME,2019-11,OCONS,B,NaN,DEFERRED,0,5638408.73,5638408.73
15640,EUR,Experience Cloud,1TME,2015-03,OCONS,A,NaN,DEFERRED,0,1500.00,1695.45
5977,CAD,Document Cloud,NaN,2016-09,1Y,D,Y3,DEFERRED,0,120.00,93.34


## Clearing out the Non-Revenue billings from the file
 - No Idea what these are

In [15]:
df["Sales Type"].value_counts()

DEFERRED       37628
RECOGNIZED      7183
PRO-SVC-INV     1323
NON-REV          150
Name: Sales Type, dtype: int64

In [16]:
print('Length of the dataframe before removing non-revenue billings: ', len(df))
df = df[df['Sales Type']!='NON-REV']
print('Length of the dataframe after removing non-revenue billings:  ', len(df))


Length of the dataframe before removing non-revenue billings:  46285
Length of the dataframe after removing non-revenue billings:   46135


## Starting to group the revenue by period, industry, etc

Attempting to group by the following categories
 - currency
 - period
 - sale type
 
May need to process the data differently with the deferred billings so we will start with the recognized and then the service billings

# DOING THIS ALL IN PANDAS WITH SPLIT APPLY COMBINE on Sales Type 


In [23]:
# First split the data into three dataframes
# Recognized billings
rec = df[df['Sales Type']=='RECOGNIZED']
svc = df[df['Sales Type']=='PRO-SVC-INV']
dfr = df[df['Sales Type']=='DEFERRED']

# NOW WORKING ON THE BILLINGS

### Recognized Revenue

In [24]:
rec.sample(10)

,curr,BU,Frequency,period,config,rev_req_type,rebill_rule,Sales Type,Subscription Term,DC_amount,US_amount
26379,INR,Creative,NaN,2019-12,NaN,NaN,NaN,RECOGNIZED,0,118953.62,1670.57
3420,AUD,Experience Cloud,ACTL,2017-04,ONORE,NaN,NaN,RECOGNIZED,0,495384.17,374233.53
48180,USD,Other Solutions,NaN,2017-09,NaN,NaN,NaN,RECOGNIZED,0,-3297.65,-3297.65
16850,EUR,Experience Cloud,CERT,2015-06,NaN,NaN,NaN,RECOGNIZED,0,237600.00,264624.62
39637,USD,Creative,ONGO,2015-06,ONORE,NaN,NaN,RECOGNIZED,0,1532.34,1532.34
48453,USD,Print & Publishing,NaN,2015-08,1Y,NaN,NaN,RECOGNIZED,0,2100.00,2100.00
16768,EUR,Experience Cloud,ACTL,2017-10,ONORE,NaN,NaN,RECOGNIZED,0,810547.39,969541.11
30691,JPY,LiveCycle,NaN,2015-06,NaN,NaN,NaN,RECOGNIZED,0,684800.00,5731.78
31158,JPY,Print & Publishing,NaN,2017-11,NaN,NaN,NaN,RECOGNIZED,0,9932227.00,88208.92
48599,USD,Print & Publishing,NaN,2016-02,2V,NaN,NaN,RECOGNIZED,0,88693.00,88693.00


In [25]:
# testing groupby object
gb_rec = rec.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [26]:
# the Subscription term hangs around. We are dropping that here
gb_rec.drop(labels='Subscription Term', axis=1,inplace =True)

In [27]:
gb_rec.sample(10)

,curr,BU,period,DC_amount,US_amount
3252,USD,Other Solutions,2017-10,-4048.51,-4048.51
466,CHF,Creative,2016-03,100903.80,101440.71
3212,USD,LiveCycle,2019-08,11517.50,11517.50
2590,SEK,Creative,2018-06,333928.60,38287.58
326,AUD,Print & Publishing,2019-09,74864.73,51472.01
275,AUD,Print & Publishing,2015-06,158831.25,125156.31
2042,JPY,LiveCycle,2016-05,1726600.00,15335.70
930,DKK,Experience Cloud,2018-10,21942.58,3434.17
2162,JPY,Print & Publishing,2018-12,9148237.00,80841.01
2945,USD,Creative,2018-09,9398021.93,9398021.93


## Now doing this for the service billings

In [29]:
# testing groupby object
gb_svc = svc.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [30]:
gb_svc.drop(labels='Subscription Term', axis=1,inplace =True)
gb_svc.head(5)

,curr,BU,period,DC_amount,US_amount
0,AUD,Experience Cloud,2015-01,25075.00,21084.51
1,AUD,Experience Cloud,2015-02,-4750.00,-4220.74
2,AUD,Experience Cloud,2015-03,424271.75,337822.37
3,AUD,Experience Cloud,2015-04,297925.00,229116.23
4,AUD,Experience Cloud,2015-05,316894.82,237310.85


# NOW WORKING ON DEFERRED BILLINGS

## Type B billings are service agreements that will have invoices submitted before the billings are reclassified to revenue. If no invoices are assigned to the billings, the billings become revenue in 12 months

In [32]:
# filter out the type B first then do a group_by
dfr_b = dfr[dfr['rev_req_type']=='B']

In [33]:
gb_b = dfr_b.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [34]:
gb_b.drop(labels='Subscription Term', axis=1, inplace=True)

In [35]:
gb_b.head(10)

,curr,BU,period,DC_amount,US_amount
0,AUD,Creative,2017-10,8544.0,6837.59
1,AUD,DX Other,2017-04,13205.0,10082.02
2,AUD,DX Other,2017-08,27400.0,21259.93
3,AUD,DX Other,2018-07,6635.4,4908.74
4,AUD,DX Other,2018-08,7535.0,5561.43
5,AUD,Document Cloud,2017-12,3283.2,2494.58
6,AUD,Document Cloud,2018-06,3425.0,2592.18
7,AUD,Document Cloud,2018-12,3425.0,2483.47
8,AUD,Document Cloud,2019-03,4375.0,3122.00
9,AUD,Experience Cloud,2015-01,157510.0,131675.74


In [36]:
print('length of deferred billings : ', len(dfr))
print('length of the type B billings: ', len(dfr_b))

length of deferred billings :  37628
length of the type B billings:  1434


## Now Type A Billings
These billings are on a billing plan. The product config tells us how long before they renew

 - '2Y' = 24 months
 - '1Y' = 12 months
 - 'MTHLY' = 1 month
 
NOTE: There are also other fields in the 'Product Configtype ID' field that do not map well to a rebill period.
To fix this, we need to load up a different file and determine the length of the sales contract (type A no config)
 

In [37]:
# filtering just the type A billings
dfr_a = dfr[dfr['rev_req_type']=='A']

In [38]:
gb_a = dfr_a.groupby(['curr', 'BU', 'period',
                     'config'], as_index=False).sum()
gb_a.drop(labels='Subscription Term', axis=1, inplace = True)

In [39]:
gb_a.sample(10)

,curr,BU,period,config,DC_amount,US_amount
4404,USD,Experience Cloud,2017-10,2Y,2046.00,2046.00
4278,USD,Experience Cloud,2015-01,1Y,2725125.59,2725125.59
4634,USD,Print & Publishing,2017-01,1Y,4468602.67,4468602.67
3746,SEK,Print & Publishing,2016-01,1Y,-514.00,-60.04
491,AUD,Print & Publishing,2015-07,2Y,6525.00,5038.13
604,BRL,Creative,2018-10,1Y,-34605.06,-8401.32
3769,SEK,Print & Publishing,2018-06,1Y,430.50,49.55
4571,USD,Other Solutions,2020-01,OUNIV,12250.00,12250.00
3239,JPY,Print & Publishing,2018-07,1Y,5813748.00,53026.17
3115,JPY,Experience Cloud,2018-01,1Y,123999.00,1100.19


In [40]:
gb_a['config'].value_counts()

1Y       2418
MTHLY     950
2Y        875
OUNIV     231
OCONS     106
3Y        101
ONORE      32
1V          1
OENSV       1
Name: config, dtype: int64

### Below is just a check to see how large the billing types are across all periods

In [41]:
gb_a_config = gb_a.groupby(['config'], as_index=False).sum()
gb_a_config

,config,DC_amount,US_amount
0,1V,6.503000e+03,6.503000e+03
1,1Y,1.198135e+11,9.655797e+09
2,2Y,2.470307e+08,6.719101e+07
3,3Y,9.380464e+08,4.051346e+06
4,MTHLY,1.381940e+08,3.985406e+07
5,OCONS,3.903233e+07,1.028252e+07
6,OENSV,-1.200000e+02,-1.200000e+02
7,ONORE,2.350904e+06,2.015485e+06
8,OUNIV,4.283761e+06,3.473767e+06


### These 'OCONS', 'ONORE' and 'OUNIV' data types are not actual product config IDs so we have to get them from a different data file. We are excluding these types below.

In [46]:
config_list = ['1Y', '2Y', '3Y', 'MTHLY']
test1 = gb_a[gb_a['config'].isin(config_list)]


In [47]:
test1.sample(5)

,curr,BU,period,config,DC_amount,US_amount
1965,EUR,Print & Publishing,2016-02,2Y,14280.00,15536.38
3689,SEK,Document Cloud,2018-08,MTHLY,-1221.38,-137.83
4147,USD,Document Cloud,2017-05,1Y,6586147.77,6586147.77
126,AUD,Creative,2018-07,2Y,10888.83,8221.41
714,CAD,Experience Cloud,2017-02,1Y,119180.55,90043.86


## For now, lets just split this into gb_a_1Y, gb_a_2Y, gb_a_3y, gb_a_1M


In [48]:
gb_a_1Y = test1[test1['config']=='1Y']
gb_a_2Y = test1[test1['config']=='2Y']
gb_a_3Y = test1[test1['config']=='3Y']
gb_a_1M = test1[test1['config']=='MTHLY']

In [49]:
print('this is the lenght of type A 1M billings: ', len(gb_a_1M))
print('this is the lenght of type A 1Y billings: ', len(gb_a_1Y))
print('this is the lenght of type A 2Y billings: ', len(gb_a_2Y))
print('this is the lenght of type A 3Y billings: ', len(gb_a_3Y))

this is the lenght of type A 1M billings:  950
this is the lenght of type A 1Y billings:  2418
this is the lenght of type A 2Y billings:  875
this is the lenght of type A 3Y billings:  101


# TYPE D billings
These billings have a field 'Rule For Bill Date' that determines when new billings will occur
 - Monthly [Y1, Y2, Y3, Y5]
 - Quarterly [YQ]
 - Every 4 months [YT]
 - Semi-annual [YH]
 - Annual [YA, YC]
 - Every 2 years - [Y4]
 
 We also need to track the type D billings that do not have a 'Rule for Bill Date'

In [51]:
# for now just do a groupby on the type
# filtering just the type A billings
dfr_d = dfr[dfr['rev_req_type']=='D']

In [52]:
gb_d = dfr_d.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [53]:
gb_d_mthly = gb_d[gb_d['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'Y5'])]
gb_d_qtrly = gb_d[gb_d['rebill_rule']=='YQ']
gb_d_four_mths = gb_d[gb_d['rebill_rule']=='YT']
gb_d_semi_ann = gb_d[gb_d['rebill_rule']=='YH']
gb_d_annual = gb_d[gb_d['rebill_rule'].isin(['YA', 'YC'])]
gb_d_two_yrs = gb_d[gb_d['rebill_rule']=='Y4']

In [54]:
gb_d['rebill_rule'].value_counts()

YA    2952
Y3    2375
YQ     991
YH     547
Y1     491
YC     490
Y2     153
Y4      57
Y7      31
YM      23
YX      12
YY      10
YT      10
Name: rebill_rule, dtype: int64

### Note: There is one type D billings that is listed as 'BT' I don't know what this means, but it was a $180 EUR payment from January 2017, so we will ignore

In [55]:
print('Length of monthly', len(gb_d_mthly))
print('Length of quarterly', len(gb_d_qtrly))
print('Length of four months', len(gb_d_four_mths))
print('Length of semi ann', len(gb_d_semi_ann))
print('Length of annual', len(gb_d_annual))
print('Length of two years', len(gb_d_two_yrs))

Length of monthly 3019
Length of quarterly 991
Length of four months 10
Length of semi ann 547
Length of annual 3442
Length of two years 57


In [56]:
what_is_YT = gb_d[gb_d['rebill_rule']=='YT']
what_is_YT.head(10)

,curr,BU,period,rebill_rule,DC_amount,US_amount
6985,USD,DX Other,2016-12,YT,13440.00,13440.00
6999,USD,DX Other,2017-04,YT,13440.00,13440.00
7011,USD,DX Other,2017-08,YT,13440.00,13440.00
7436,USD,Experience Cloud,2015-02,YT,612916.60,612916.60
7444,USD,Experience Cloud,2015-03,YT,698888.89,698888.89
7465,USD,Experience Cloud,2015-06,YT,11666.60,11666.60
7480,USD,Experience Cloud,2015-08,YT,549861.11,549861.11
7495,USD,Experience Cloud,2015-10,YT,11666.60,11666.60
7518,USD,Experience Cloud,2016-01,YT,612916.60,612916.60
7526,USD,Experience Cloud,2016-02,YT,-601250.00,-601250.00


## Note: There are only 8 payments (back from 2017) that paid on a 4 month basis. 
We will ignore these as well.

## NOW WE NEED TO BUILD A DATAFRAME THAT INTEGRATES THIS DATA

- We will have the following descriptive fields
   - Invoicing Fiscal Year-Period
   - Document Currency
   - Enterprise BU

- We will have the following fields based on rebilling rule
   - Recognized
   - Service
   - Monthly
   - Quarterly
   - Annual
   - Two Years
   - Three Years

In [57]:
# We need to do it this way when we get to a .py file!
list_df = [gb_rec, gb_svc, gb_b,
        gb_a_1M,    gb_a_1Y,    gb_a_2Y,       gb_a_3Y, 
        gb_d_mthly, gb_d_qtrly, gb_d_semi_ann, gb_d_annual, gb_d_two_yrs]

list_columns = ['recognized', 'service', 'deferred_B', 
    'deferred_1M_a', 'deferred_1Y_a', 'deferred_2Y_a', 'deferred_3Y_a',
    'deferred_1M_d', 'deferred_3M_d', 'deferred_6M_d', 'deferred_1Y_d', 'deferred_2Y_d']


In [64]:
def merge_new_dataframe(old_df, new_df, new_column):
    df_merged = pd.merge(old_df, new_df, how='outer', 
                     left_on=['curr', 'BU', 'period'],
                    right_on=['curr', 'BU', 'period'])
    df_merged.rename(index=str, columns={'DC_amount': new_column}, inplace=True)
    
    #need to drop the product configtype id for merges where the new_df is of type A
    config_str = 'config'
    rule_str = 'rebill_rule'
    if config_str in df_merged.columns:
        df_merged.drop(columns=['config'], inplace=True)
    
    if rule_str in df_merged.columns:
        df_merged.drop(columns=['rebill_rule'], inplace=True)
        
    return df_merged

In [65]:
def merge_all_dataframes(list_df, list_columns):
    for i, df in enumerate(list_df):
        print('This is i:', i)
        #print("This is the df: ", df.head())
        print('referencing the column: ', list_columns[i])

        if i==0:
            df_merged = list_df[0]
            df_merged.rename(index=str, columns={'amount': list_columns[i]}, inplace=True)
        else:
            df_merged = merge_new_dataframe(df_merged, df, list_columns[i])

    return df_merged
    

In [66]:
df = merge_all_dataframes(list_df, list_columns)

This is i: 0
referencing the column:  recognized
This is i: 1
referencing the column:  service
This is i: 2
referencing the column:  deferred_B
This is i: 3
referencing the column:  deferred_1M_a
This is i: 4
referencing the column:  deferred_1Y_a
This is i: 5
referencing the column:  deferred_2Y_a
This is i: 6
referencing the column:  deferred_3Y_a
This is i: 7
referencing the column:  deferred_1M_d
This is i: 8
referencing the column:  deferred_3M_d
This is i: 9
referencing the column:  deferred_6M_d
This is i: 10
referencing the column:  deferred_1Y_d
This is i: 11
referencing the column:  deferred_2Y_d


In [69]:
df.columns

Index(['curr', 'BU', 'period', 'DC_amount_x', 'US_amount_x', 'DC_amount_y',
       'US_amount_y', 'deferred_B', 'US_amount_x', 'deferred_1M_a',
       'US_amount_y', 'deferred_1Y_a', 'US_amount_x', 'deferred_2Y_a',
       'US_amount_y', 'deferred_3Y_a', 'US_amount_x', 'deferred_1M_d',
       'US_amount_y', 'deferred_3M_d', 'US_amount_x', 'deferred_6M_d',
       'US_amount_y', 'deferred_1Y_d', 'US_amount_x', 'deferred_2Y_d',
       'US_amount_y'],
      dtype='object')

In [68]:
def clean_df_columns(df):
    
    # clean up NaNs before adding 
    df = df.fillna(value=0)
    
    # Monthly
    df['deferred_1M'] = df['deferred_1M_a']+df['deferred_1M_d']
    df.drop(labels=['deferred_1M_a', 'deferred_1M_d'], axis=1, inplace=True)
    
    # Annual
    df['deferred_1Y'] = df['deferred_1Y_a']+df['deferred_1Y_d']
    df.drop(labels=['deferred_1Y_a', 'deferred_1Y_d'], axis=1, inplace=True)
    
    # Two-Year
    df['deferred_2Y'] = df['deferred_2Y_a']+df['deferred_2Y_d']
    df.drop(labels=['deferred_2Y_a', 'deferred_2Y_d'], axis=1, inplace=True)
    
    # renaming 3Y, 3M and 6M
    df.rename(index=str, columns = {'deferred_3Y_a':'deferred_3Y', 'deferred_3M_d':'deferred_3M', 
                               'deferred_6M_d': 'deferred_6M'}, inplace=True)

    #cleaning up the longer column names
    df.rename(index=str, columns = {'curr': 'curr',
                               'BU':'BU',
                               'period':'period'}, inplace=True)
    
    return df

In [63]:
df = clean_df_columns(df)

KeyError: 'deferred_1M_a'

In [ ]:
df.sample(10)

In [ ]:
# Make this a function to be cleaned up somehow
del dfr
del dfr_a
del dfr_b
del dfr_d
del gb_a
del gb_a_1M
del gb_a_1Y
del gb_a_2Y
del gb_a_3Y
del gb_b, 
del gb_d
del gb_svc, gb_rec, gb_d_two_yrs
del gb_d_four_mths, gb_d_qtrly, gb_d_semi_ann


# Now working on the ZCC billings

These billings are type D billings that did not populate the rebill_rule field of the database.

They have a 'sales document type' = 'ZCC"

The billings themselves are being created from a tableau report that looks for additions to the deferred revenue waterfall based on billings of type D and have a sales document type of ZCC

In [ ]:
whos

In [ ]:
df.sample(10)

# TO BE DONE:

1. Clean up the type F billings (at least check to see if they are necessary)
2. Make a function to delete all intermediate dataframes
3. Add type A no config function
4. Add type D ZCC billings

5. Work on the forecast part of this

6. Load up FX rates

In [ ]:
# Adobe financial calendar
df_cal = pd.read_excel('../data/old/ADOBE_FINANCIAL_CALENDAR.xlsx', 'ADBE_cal')

In [ ]:
df_cal.head()

## ZCC Billings

In [ ]:
df_ZCC = pd.read_excel('../data/old/type_D_ZCC_billings.xlsx', sheet_name='DC')

In [ ]:
df_ZCC.sample(10)

### Renaming the columns

In [ ]:
df_ZCC.columns

In [ ]:
df_ZCC.rename(index = str, columns = {'Document Currency': 'curr',
                                      'Enterprise BU Description':'BU',
                                      'Rule for Bill Date Code': 'rebill_rule',
                                      'Week of FICA Posting Date (YYYYMMDD) (copy)': 'fiscal_week',
                                      'DF Additions - Doc Curr': 'amount'}, inplace = True)

In [ ]:
df_ZCC.columns

### Checking that we do not have any currencies that need to be removed


In [ ]:
df_ZCC['curr'].value_counts()

In [ ]:
model_dict

In [ ]:
ZCC_curr = df_ZCC['curr'].unique()

In [ ]:
for item in ZCC_curr:
    if item in model_dict['curr_removed']:
        print('This currency needs to be removed: ', item)

There are no currencies that need to be removed

# Now we are clearing out nans

In [ ]:
print('Number of NaNs to be cleared out: ', sum(df_ZCC['amount'].isna()))
ZCC_isna = df_ZCC['amount'].isna()
print("This is the length of the ZCC records before clearing NAs: ", len(df_ZCC))
df_ZCC = df_ZCC[~ZCC_isna]
print("This is the length of the ZCC records before clearing NAs: ", len(df_ZCC))

In [ ]:
df_cal.sample(5)

In [ ]:
df_ZCC.sample(10)

In [ ]:
def add_ZCC_periods(df_ZCC, df_cal):
    ''' 
    Takes each ZCC period billing and maps it to the fiscal calendar then assignes the
    ZCC billing to the dataframe df
    '''
    period_list = []
    
    for i in range(len(df_ZCC)):
        # this_date is the date we need mapped to a period
        this_date = df_ZCC['fiscal_week'].iloc[i]
        
        #max index will be the index to the period in our financial calendar
        this_index = df_cal['Per_End']<=this_date
        max_index = sum(this_index)

        this_year = df_cal['Year'].iloc[max_index].astype(str)
        this_period= df_cal['Period'].iloc[max_index].astype(str)

        # formatting the period string from the calendar
        if len(this_period)== 1:
            this_period = '0'+this_period

        period_ticker = this_year + '-' + this_period

        period_list.append(period_ticker)

    df_ZCC['period'] = period_list
    return df_ZCC


In [ ]:
df_ZCC.head()

In [ ]:
df_ZCC = add_ZCC_periods(df_ZCC, df_cal)

In [ ]:
df_ZCC.tail()

## Now we need to determine the length of the ZCC billings to and add this to our dataframe

In [ ]:
df.columns

In [ ]:
df_ZCC.columns

In [ ]:
gb_ZCC = df_ZCC.groupby(['curr', 'BU', 'period',
                     'rebill_rule'], as_index=False).sum()
#gb_d.drop(labels='Subscription Term', axis=1, inplace = True)

In [ ]:
gb_ZCC.sample(20)

In [ ]:
gb_ZCC['rebill_rule'].value_counts()

In [ ]:
gb_ZCC_mthly = gb_ZCC[gb_ZCC['rebill_rule'].isin(['Y1', 'Y2', 'Y3', 'Y5'])]
gb_ZCC_qtrly = gb_ZCC[gb_ZCC['rebill_rule']=='YQ']
gb_ZCC_four_mths = gb_ZCC[gb_ZCC['rebill_rule']=='YT']
gb_ZCC_semi_ann = gb_ZCC[gb_ZCC['rebill_rule']=='YH']
gb_ZCC_annual = gb_ZCC[gb_ZCC['rebill_rule'].isin(['YA', 'YC'])]
gb_ZCC_two_yrs = gb_ZCC[gb_ZCC['rebill_rule']=='Y4']

There are no four month billings

In [ ]:
len(gb_ZCC_four_mths)

There are two YX billings for small amounts that I do not have mapped to a frequency.
They will be ignored

In [ ]:
what_is_YX = gb_ZCC[gb_ZCC['rebill_rule']=='YX']
what_is_YX.head(10)

## I need to merge the ZCC billings and then clean up the columns. 
 


In [ ]:
# We need to do it this way when we get to a .py file!
list_ZCC_df = [gb_ZCC_mthly, gb_ZCC_qtrly, gb_ZCC_semi_ann,
           gb_ZCC_annual, gb_ZCC_two_yrs]

list_ZCC_columns = ['deferred_1M', 'deferred_3M', 'deferred_6M', 
    'deferred_1Y', 'deferred_2Y']


In [ ]:
df_ZCC_merged = merge_all_dataframes(list_ZCC_df, list_ZCC_columns)

In [ ]:
df_ZCC_merged.sample(10)

### Clearing out any zeros

In [ ]:
df_ZCC_merged.drop(columns=['rebill_rule_x', 'rebill_rule_y'], inplace=True)
df_ZCC_merged.fillna(0, inplace=True)
df_ZCC_merged.sample(10)

In [ ]:
df.columns

In [ ]:
df_cal.sample(5)

In [ ]:
test_merge = pd.merge(df, df_ZCC_merged, how='outer', 
                 left_on=['curr', 'BU', 'period'],
                right_on=['curr', 'BU', 'period'])
#test_merge.rename(index=str, columns={'amount': new_column}, inplace=True)


In [ ]:
test_merge

In [ ]:

# clean up NaNs before adding 
test_merge = test_merge.fillna(value=0)

# Monthly
test_merge['deferred_1M'] = test_merge['deferred_1M_x']+test_merge['deferred_1M_y']
test_merge.drop(labels=['deferred_1M_x', 'deferred_1M_x'], axis=1, inplace=True)

# Quarterly
test_merge['deferred_3M'] = test_merge['deferred_3M_x']+test_merge['deferred_3M_y']
test_merge.drop(labels=['deferred_3M_x', 'deferred_3M_y'], axis=1, inplace=True)

# Semi-Annual
test_merge['deferred_6M'] = test_merge['deferred_6M_x']+test_merge['deferred_6M_y']
test_merge.drop(labels=['deferred_6M_x', 'deferred_6M_y'], axis=1, inplace=True)

# Annual
test_merge['deferred_1Y'] = test_merge['deferred_1Y_x']+test_merge['deferred_1Y_y']
test_merge.drop(labels=['deferred_1Y_x', 'deferred_1Y_y'], axis=1, inplace=True)

# 2-Years
test_merge['deferred_2Y'] = test_merge['deferred_2Y_x']+test_merge['deferred_2Y_y']
test_merge.drop(labels=['deferred_2Y_x', 'deferred_2Y_y'], axis=1, inplace=True)





In [ ]:
test_merge.columns

In [ ]:
test_merge.sample(20)

In [ ]:
## Looks like the type ZCC billings are complete

In [ ]:
# renaming the main dataframe as df
df = test_merge.copy()

# Type A No Config Type Billings

This file contains type A billings that have a revenue contract start date and end date. We need to map these into the terms of our dataframe.

### Steps:
1. Rename the columns
2. This file has entries for pennies. Need to clear out anything less than $10 in absolute value
3. Determine the length of time between start date and end date
4. Group this dataframe by currency, period and BU
5. Merge this final dataframe with the larger dataframe

## NOTE: This file contains two different start date and end date columns. We need to look at all of them

In [ ]:
df_A = pd.read_excel('../data/old/type_A_no_config.xlsx', 'DC')

In [ ]:
df_A.sample(10)

In [ ]:
df_A.columns

In [ ]:
df_A.rename(index=str, columns={'Document Currency':'currency', 
                               'Enterprise Bu':'BU',
                               'Invoicing Fiscal Year-Period Desc':'period',
                               'Rev Rec Contract End Date Hdr':'end_date_1',
                               'Rev Rec Contract End Date Item':'end_date_2',
                               'Rev Rec Contract Start Date Hdr': 'start_date_1',
                               'Rev Rec Contract Start Date Item': 'start_date_2',
                               'Completed Sales Doc Currency':'amount'}, inplace=True)

df_A.drop(columns='Product Configtype ID', inplace=True)

In [ ]:
df_A.columns

### Dealing with the duplicate dates by taking a max

In [ ]:
df_A.head()

In [ ]:
df_A['start_date_str'] = df_A[['start_date_1','start_date_2']].max(axis=1).astype(int).astype(str)
df_A['end_date_str'] = df_A[['end_date_1','end_date_2']].max(axis=1).astype(int).astype(str)

In [ ]:
df_A.sample(10)

In [ ]:
df_A['start_date'] = pd.to_datetime(df_A['start_date_str'])
df_A['end_date'] = pd.to_datetime(df_A['end_date_str'])

In [ ]:
df_A.drop(labels=['end_date_1', 'end_date_2', 'start_date_1', 'start_date_2',
                  'start_date_str', 'end_date_str'], axis=1, inplace=True)

In [ ]:
df_A['month_interval']=(df_A['end_date']-df_A['start_date'])

In [ ]:
df_A.sample(10)

In [ ]:
df_A['months']= (df_A['month_interval']/ np.timedelta64(1,'M')).round(0)

### Now I need to map the months into the different integers in my dataframe


In [ ]:
list_rebills = [1, 3, 6, 12, 24, 36]
temp_rebill = np.zeros_like(df_A['months'])
for i in range(len(df_A)):
    temp_rebill[i] = min(list_rebills, key=lambda x:abs(x-df_A['months'][i]))
df_A['rebill_months']=temp_rebill


In [ ]:
df_A.sample(10)

In [ ]:
plt.scatter(df_A['months'], df_A['rebill_months'])

## Grouping the dataframe by rebill_months

In [ ]:
# drop what we dont need
df_A.drop(columns = ['start_date', 'end_date', 'month_interval', 'months'], axis=1, inplace=True)

In [ ]:
df_A.head()

In [ ]:
#gb_A = df+A.groupby(['curr', 'BU', 'period'], as_index=False).sum()

In [ ]:
#medals = df.pivot_table('no of medals', ['Year', 'Country'], 'medal')
temp = df_A.pivot_table('amount', ['currency', 'BU', 'period'], 'rebill_months')

In [ ]:
temp

In [ ]:
temp.columns[5]

In [ ]:
temp.fillna(0)

In [ ]:
temp.rename(index=str, columns = {''})

In [ ]:
whos